In [2]:
import pandas as pd
from scipy.stats import skew, kurtosis
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
df = pd.read_csv('game-weather.csv')
print(df)

    dia       tempo  temperatura  umidade vento joga
0    d1     chuvoso           71       91   sim  nao
1    d2  ensolarado           69       70   nao  sim
2    d3  ensolarado           80       90   sim  nao
3    d4     nublado           83       86   nao  sim
4    d5     chuvoso           70       96   nao  sim
5    d6     chuvoso           65       70   sim  nao
6    d7     nublado           64       65   sim  sim
7    d8     nublado           72       90   sim  sim
8    d9  ensolarado           75       70   sim  sim
9   d10     chuvoso           68       80   nao  sim
10  d11     nublado           81       75   nao  sim
11  d12  ensolarado           85       85   nao  nao
12  d13  ensolarado           72       95   nao  nao
13  d14     chuvoso           75       80   nao  sim


In [4]:
contagem = df['joga'].value_counts()
print(contagem)
porcentagem = contagem / contagem.sum() * 100
# Identificar a maioria e a minoria
maioria = porcentagem.idxmax()
minoria = porcentagem.idxmin()

print(f'Maioria: {maioria} ({porcentagem.max():.2f}%)')
print(f'Minoria: {minoria} ({porcentagem.min():.2f}%)')

joga
sim    9
nao    5
Name: count, dtype: int64
Maioria: sim (64.29%)
Minoria: nao (35.71%)


In [5]:
# Selecionar apenas as colunas numéricas
atributos_quantitativos = df.select_dtypes(include=['float64', 'int64'])

# Calcular o Q1 (1º quartil) e Q3 (3º quartil)
Q1 = atributos_quantitativos.quantile(0.25)
Q3 = atributos_quantitativos.quantile(0.75)

# Calcular o IQR
IQR = Q3 - Q1

# Definir os limites para outliers
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Verificar se existem valores abaixo do limite inferior ou acima do limite superior
outliers = (atributos_quantitativos < limite_inferior) | (atributos_quantitativos > limite_superior)
print(outliers)

# Mostrar os outliers
outliers_detectados = atributos_quantitativos[outliers.any(axis=1)]
print(outliers_detectados)

    temperatura  umidade
0         False    False
1         False    False
2         False    False
3         False    False
4         False    False
5         False    False
6         False    False
7         False    False
8         False    False
9         False    False
10        False    False
11        False    False
12        False    False
13        False    False
Empty DataFrame
Columns: [temperatura, umidade]
Index: []


In [6]:
# Considere que um novo exemplo será colocado no conjunto de dados, em que a umidade é de 85, há vento e a pessoa joga. 
# Os atributos tempo e temperatura são desconhecidos. 
# Estime os valores desconhecidos para esse novo exemplo, usando alguma estratégia de preenchimento. Justifique sua resposta.

# Preencher o valor desconhecido do atributo 'tempo' com a moda
tempo_moda = df['tempo'].mode()[0]

# Preencher o valor desconhecido do atributo 'temperatura' com a moda
temperatura_moda = df['temperatura'].mode()[0]

print(f'Tempo estimado: {tempo_moda}')
print(f'Temperatura estimada: {temperatura_moda}')

Tempo estimado: chuvoso
Temperatura estimada: 72


In [7]:
# Preencher o valor desconhecido do atributo 'temperatura' com a média
temperatura_media = df['temperatura'].mean()

print(f'Temperatura estimada: {temperatura_media}')


Temperatura estimada: 73.57142857142857


In [8]:
# Definindo as categorias para temperatura
def categorizar_temperatura(temp):
    if temp < 15:
        return 'fria'
    elif 15 <= temp <= 25:
        return 'média'
    else:
        return 'quente'

# Aplicando a função no DataFrame
df['temp_discretizada'] = df['temperatura'].apply(categorizar_temperatura)

# Definindo as categorias para umidade
def categorizar_umidade(umid):
    if umid > 75:
        return 'alta'
    else:
        return 'normal'

# Aplicando a função no DataFrame
df['umidade_discretizada'] = df['umidade'].apply(categorizar_umidade)

pd.set_option('display.max_columns', None)  # Mostra todas as colunas sem limitação
pd.set_option('display.expand_frame_repr', False)  # Evita a quebra de linha para novas colunas
print(df)

    dia       tempo  temperatura  umidade vento joga temp_discretizada umidade_discretizada
0    d1     chuvoso           71       91   sim  nao            quente                 alta
1    d2  ensolarado           69       70   nao  sim            quente               normal
2    d3  ensolarado           80       90   sim  nao            quente                 alta
3    d4     nublado           83       86   nao  sim            quente                 alta
4    d5     chuvoso           70       96   nao  sim            quente                 alta
5    d6     chuvoso           65       70   sim  nao            quente               normal
6    d7     nublado           64       65   sim  sim            quente               normal
7    d8     nublado           72       90   sim  sim            quente                 alta
8    d9  ensolarado           75       70   sim  sim            quente               normal
9   d10     chuvoso           68       80   nao  sim            quente          

In [9]:
# Função para calcular a entropia
def calcular_entropia(coluna):
    proporcoes = coluna.value_counts(normalize=True)
    return -sum(proporcoes * np.log2(proporcoes))

# Entropia da classe alvo (Joga)
entropia_joga = calcular_entropia(df['joga'])

# Função para calcular o ganho de informação de um atributo
def ganho_informacao(df, atributo, alvo):
    valores = df[atributo].unique()
    entropia_condicional = 0
    for valor in valores:
        subset = df[df[atributo] == valor]
        proporcao = len(subset) / len(df)
        entropia_condicional += proporcao * calcular_entropia(subset[alvo])
    return entropia_joga - entropia_condicional

# Calculando o ganho de informação para cada atributo
ganho_temp = ganho_informacao(df, 'temp_discretizada', 'joga')
ganho_umidade = ganho_informacao(df, 'umidade_discretizada', 'joga')

# Resultado
print(f"Ganho de Informação - Temperatura: {ganho_temp}")
print(f"Ganho de Informação - Umidade: {ganho_umidade}")

# Ranquear os atributos
ranking = {
    'temp_discretizada': ganho_temp,
    'umidade_discretizada': ganho_umidade
}

ranking_ordenado = sorted(ranking.items(), key=lambda x: x[1], reverse=True)
print("\nRanking dos Atributos:")
for atributo, ganho in ranking_ordenado:
    print(f"{atributo}: {ganho}")
    

Ganho de Informação - Temperatura: 0.0
Ganho de Informação - Umidade: 0.04533417202914447

Ranking dos Atributos:
umidade_discretizada: 0.04533417202914447
temp_discretizada: 0.0
